# Lession 21 - (LeNet for MNIST)

In [40]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.callbacks import EarlyStopping
# New
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16
# from keras.optimizers import Adam
import matplotlib.pyplot as plt
from time import time
%matplotlib inline

In [13]:
MNIST = np.load('../data/MNIST_train_1000.npz')
images = MNIST['train_images']
labels = MNIST['train_labels']
print(images.shape)
print(labels.shape)

(1000, 28, 28)
(1000,)


Need to add extra dimension for B&W images to make it a 4D tensor

In [17]:
X = np.expand_dims(images, -1)
print(X.shape)
P = pd.get_dummies(pd.DataFrame(labels, columns=['digits'], dtype='category')).values
print(P.shape)


(1000, 28, 28, 1)
(1000, 2)


Does it need scaling?  No

In [19]:
X.max()

1.0

In [32]:
# build LeNet
model = Sequential()
def BuildLeNet(model, input_shape=(32, 32, 3), outputs=10):
    # 20 channels out, 5 x 5 kernel, keep size the same
    model.add(Conv2D(20, 5, padding='same', input_shape=input_shape))
    model.add(Activation('relu'))
    # pool_size = size of pooling operation, strides=2 - Downsample by 2 in each directions
    model.add(MaxPooling2D(pool_size=2, strides=2))
    model.add(Conv2D(50, 5, padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=2, strides=2))
    model.add(Flatten())
    model.add(Dense(500, activation='relu'))
    model.add(Dense(2,   activation='softmax'))
    return model

LeNet = BuildLeNet(model, input_shape=(28, 28, 1), outputs=2)
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 28, 28, 20)        520       
_________________________________________________________________
activation_13 (Activation)   (None, 28, 28, 20)        0         
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 14, 14, 20)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 14, 14, 50)        25050     
_________________________________________________________________
activation_14 (Activation)   (None, 14, 14, 50)        0         
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 7, 7, 50)          0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 2450)              0         
__________

In [37]:
LeNet.compile(loss='categorical_crossentropy',
             optimizer='Adam',
             metrics=['accuracy'])
hist = LeNet.fit(X, P, epochs=100, validation_split=0.2, verbose=1,
                callbacks=[EarlyStopping(patience=3)])

Train on 800 samples, validate on 200 samples
Epoch 1/100
800/800 [==============================] - 1s 2ms/step - loss: 0.0137 - acc: 0.9950 - val_loss: 0.1052 - val_acc: 0.9750
Epoch 2/100
800/800 [==============================] - 1s 855us/step - loss: 0.0121 - acc: 0.9937 - val_loss: 0.0685 - val_acc: 0.9900
Epoch 3/100
800/800 [==============================] - 1s 838us/step - loss: 0.0027 - acc: 0.9988 - val_loss: 0.1105 - val_acc: 0.9800
Epoch 4/100
800/800 [==============================] - 1s 841us/step - loss: 0.0013 - acc: 1.0000 - val_loss: 0.0919 - val_acc: 0.9850
Epoch 5/100
800/800 [==============================] - 1s 913us/step - loss: 2.2450e-04 - acc: 1.0000 - val_loss: 0.1078 - val_acc: 0.9800
